Neural Network

In [ ]:
from ucimlrepo import fetch_ucirepo
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import utils

covertype = fetch_ucirepo(id=31)

X = covertype.data.features
y = covertype.data.targets

Clean, Balance, and Split Data

In [ ]:
# preprocess data using utils.py
X_clean, y_clean = utils.clean_data(X, y)

X_train, X_test, y_train, y_test = utils.split_data(X_clean, y_clean)

# create a validation set
validation_ratio = 0.1

X_train, X_val, y_train, y_val = utils.split_data(X_train, y_train, validation_ratio)

Create Model

In [ ]:
# This model will follow the class implementation
model = Sequential([
    InputLayer(input_shape=(54,1)),
    Flatten(),      # ensure our data is flat
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')  # 5 categories
])

Compile our new model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Summary of Data
model.summary()

Training of Model

In [ ]:
fitted = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=256,
    verbose=1
)